Collab Stuff

In [ ]:
!pip install -U hbp_neuromorphic_platform

In [ ]:
username = ''
password = ''

In [ ]:
import nmpi
client = nmpi.Client(username, password)
print(client.my_collabs().keys())

In [ ]:
aistate_collab_id = client.my_collabs()['UH AI state']['id']

Pooled Version

In [ ]:
def pooled_grid_search(command):  
    job_path = client.submit_job(source="https://github.com/sesutton/GridSearch",
                              platform=nmpi.SPINNAKER,
                              collab_id=aistate_collab_id,
                              command=command) 

First grid search of conductances

In [ ]:
commands = []
for Gexc in range(1,101, 4):
    for Ginh in range(1,101, 4):
        pconn = 0.02
        n = 1500
        tstop = 60 * 1000
        commands.append("GridSearch.py {system} %s %s %s %s %s " % (Gexc, Ginh, pconn, n, tstop))

print len(commands)

In [ ]:
import multiprocessing
p = multiprocessing.Pool()
p.map(pooled_grid_search, commands)

Re-run Errors

In [ ]:
def pooled_error_check(job_id):  
    ret = client.get_job(job_id, with_log=True)
    if ret['status'] == 'error':
        return job_id

In [ ]:
first = 0
last = 0
ids = range(first, last + 1)

import multiprocessing
p = multiprocessing.Pool()
errors = p.map(pooled_error_check, ids)


In [ ]:
commands = []
for error in errors:
    if(type(error) is not int):
        continue
    ret = client.get_job(error, with_log=True)
    commands.append(ret['command'])

import multiprocessing
p = multiprocessing.Pool()
p.map(pooled_grid_search, commands)

Pooled Download Results

In [ ]:
def pooled_job_download(job_id):
    try:
        ret = client.get_job(job_id, with_log=True)
    except Exception as e:
        return str(job_id) + " " + str(e)
    status = ret['status']
    if status == 'error':
        return str(job_id) + " error"
    
    command = ret['command']
    command = command.split(' ')
    name = "Gexc %s Ginh %s pconn %s n %s tstop %s" % (str(command[2]), str(command[3]), str(command[4]), str(command[5]), str(command[6]))
    results_dir = os.getcwd() + '/results/'
    sim_dir = results_dir + "/" + name + "/"
    if not os.path.exists(sim_dir):
            os.makedirs(sim_dir)
 
    filenames = client.download_data(ret, local_dir=sim_dir)
    return str(job_id) + " success"

In [ ]:
import os

results_dir = os.getcwd() + '/results/'
if not os.path.exists(results_dir):
    os.makedirs(results_dir)


In [ ]:
first = 0
last = 0
ids = range(first, last + 1)

import multiprocessing
p = multiprocessing.Pool()
p.map(pooled_job_download, ids)

Remove Jobs from Collab

In [ ]:
def remove_comp_job(job_id):
    try:
        client.remove_completed_job(job_id)
        print "success %s" % (job_id)
    except:
        failed.append(job_id)
        print "fail %s" % (job_id)

def remove_queued_job(job_id):
    try:
        client.remove_queued_job(job_id)
        print "success %s" % (job_id)
    except:
        print "fail %s" % (job_id)
      

In [ ]:
failed = []
first = 0
last = 0

import multiprocessing
p = multiprocessing.Pool()
p.map(remove_comp_job, range(first, last + 1))
p.map(remove_queued_job, range(first, last + 1))